In [ ]:
from numba import cuda
import numpy as np
import math

@cuda.jit
def monte_carlo_update(lattice, J, T, states):
    """
    Perform Monte Carlo updates on the Ising lattice using CUDA.
    
    lattice: 2D lattice of spins (NxN array).
    J: Exchange energy.
    T: Temperature.
    states: Random states for RNG.
    """
    # Determine the position of the current thread in the grid
    x, y = cuda.grid(2)
    N = lattice.shape[0]
    
    if x < N and y < N:
        # Calculate the sum of nearest neighbors
        top = lattice[(x-1) % N, y]
        bottom = lattice[(x+1) % N, y]
        left = lattice[x, (y-1) % N]
        right = lattice[x, (y+1) % N]
        sum_neighbors = top + bottom + left + right
        
        # Calculate energy change if this spin is flipped
        delta_E = 2 * J * lattice[x, y] * sum_neighbors
        
        # Metropolis criterion
        if delta_E < 0 or math.exp(-delta_E / T) > states[x, y]:
            lattice[x, y] *= -1

def run_simulation(N, J=1.0, T=2.0, mc_steps=100):
    # Initialize the lattice and random states
    lattice = np.random.choice([-1, 1], size=(N, N)).astype(np.int32)
    states = np.random.rand(N, N).astype(np.float32)
    
    # Copy lattice to device (GPU memory)
    lattice_global_mem = cuda.to_device(lattice)
    states_global_mem = cuda.to_device(states)
    
    # Define block and grid sizes
    threadsperblock = (16, 16)
    blockspergrid_x = math.ceil(lattice.shape[0] / threadsperblock[0])
    blockspergrid_y = math.ceil(lattice.shape[1] / threadsperblock[1])
    blockspergrid = (blockspergrid_x, blockspergrid_y)
    
    # Run Monte Carlo updates
    for _ in range(mc_steps):
        monte_carlo_update[blockspergrid, threadsperblock](lattice_global_mem, J, T, states_global_mem)
    
    # Copy the lattice back to host
    lattice = lattice_global_mem.copy_to_host()
    return lattice



In [ ]:
# Example usage
N = 32  # Lattice size
final_lattice = run_simulation(N)